# Установка библиотек и их зависимостей

Установка зависимостей используемых модулей.

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    !pip install -q numpy>=1.18.5
    !pip install -q pandas>=1.0.5
    !pip install -q seaborn>=0.9.0
    !pip install -q matplotlib>=2.1.0
    !pip install -q scikit-learn>=0.23.2
    !pip install -q ucimlrepo>=0.0.7
    !pip install -q scipy>=1.14.1
    !pip install -q tqdm>=4.66.5
    !pip install -q pprintpp>=0.4.0

Необходимые ``import``'ы для выполнения задания.

In [ ]:
# Essential
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import sklearn as sk
import scipy as scp

# Scipy
from scipy.spatial.distance import cdist

# Scikit-learn preprocessing
from sklearn.preprocessing import OneHotEncoder, StandardScaler
# Scikit-learn models
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
# Scikit-learn model selection
from sklearn.model_selection import train_test_split, ParameterGrid, GridSearchCV
# Scikit-learn metrics
from sklearn.metrics import mean_absolute_error, root_mean_squared_error

# Utilities
from ucimlrepo import fetch_ucirepo
from copy import deepcopy
from tqdm.notebook import tqdm
from zlib import crc32
from pprint import pprint

# Генератор задания

Генератор задания, взятый по [этой ссылке](https://github.com/andriygav/MachineLearningSeminars/blob/master/hometask/task1-1/generator.ipynb).

In [ ]:
types = ['regression', 'classification']
datasets = {'regression': [{'name': 'Servo Data Set',
                            'url': 'https://archive.ics.uci.edu/ml/datasets/Servo'},
                           {'name': 'Forest Fires Data Set',
                            'url': 'https://archive.ics.uci.edu/ml/datasets/Forest+Fires'},
                           {'name': 'Boston Housing Data Set',
                            'url': 'https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html#sklearn.datasets.load_boston'},
                           {'name': 'Diabetes Data Set',
                            'url': 'https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html#sklearn.datasets.load_diabetes'}],
            'classification': [{'name': 'Spambase Data Set',
                                'url': 'https://archive.ics.uci.edu/ml/datasets/Spambase'},
                               {'name': 'Wine Data Set',
                                'url': 'https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html#sklearn.datasets.load_wine'},
                               {'name': 'Breast Cancer Data Set',
                                'url': 'https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html#sklearn.datasets.load_breast_cancer'},
                               {'name': 'MNIST',
                                'url': 'http://yann.lecun.com/exdb/mnist/'}]}
methods = {'regression': ['Линейная регрессия',
                          'Перцептрон',
                          'Надарая-Ватсона',
                          'SVR'],
           'classification': ['Логистическая регрессия',
                              'Перцептрон',
                              'k-ближайших соседей',
                              'Метод потенциальных функций',
                              'Метод Парзеновского окна',
                              'SVM']}
task = dict()
task['mail'] = "subkhankulov.rr@phystech.edu"
task['id'] = crc32(task['mail'].encode('utf-8'))
np.random.seed(task['id'])
task['type'] = np.random.choice(types)
task['dataset'] = np.random.choice(datasets[task['type']])
task['method'] = np.random.choice(
    methods[task['type']], size=3, replace=False).tolist()

task

# Формулировка задания
Требуется:
- Провести анализ выборки:
  - Определить тип признаков.
  - Выполнить визуальный анализ данных.
- Выполнить препроцесинг данных:
  - Преобразовать категориальные признаки в вещественные.
  - Отнормировать признаки.
- Провести эксперимент для предложенных методов (Перцептрон, Надарая-Ватсона, SVR):
  - Выполнить подбор гиперпараметров.
  - Подобрать регуляризаторы.
  - Получить итоговые модели.
- Описать полученные результаты:
  - Какая модель лучше и почему.
  - С какими проблемами столкнулись во время выполнения, возможно недочеты стандартных библиотек.
  - Совпадают ли полученные результаты с ожидаемыми результатами.


# Загрузка данных

In [ ]:
# Получаем датасет
forest_fires = fetch_ucirepo(id=162)

# Вывод метаданных
pprint(forest_fires.metadata)

In [ ]:
# Признаки
pprint(forest_fires.data.features)

In [ ]:
# Предсказываемое значение
pprint(forest_fires.data.targets)

In [ ]:
# Датасет в качестве ``pandas DataFrame``
df = forest_fires.data.features
df['target'] = forest_fires.data.targets
df.info()

# Aнализ выборки:

In [ ]:
# Информация о переменных в датасете
pprint(forest_fires.variables)

- В выборке имеем 517 объектов, 12 признаков.
- ``missing_values`` для всех признаков имеет значение 0, следовательно все признаки определены для каждого объекта в выборке.
- Имеем признаки различных типов - представленные целочисленными и действительными значениями, а также категориальные признаки:
  - Целочисленные: ``X``, ``Y``, ``DMC``, ``RH``, ``rain``.
  - Действительные значения: ``FFMC``, ``DS``, ``ISI``, ``temp``,``wind``.
  - Категориальные: ``month`` и ``day``.
- Предсказываемое значение (``target``) ``area`` является непрерывной величиной. Имеем задачу регрессии. 

In [ ]:
# Отделяем численные и категориальные признаки
categorical_features = ["month", "day"]
numerical_features = list(set(df.columns) - set(categorical_features) - set(["target"]))

print(f"Категориальные признаки: {categorical_features}")
print(f"Численные признаки: {numerical_features}")

In [ ]:
# Случайный срез из данных
df.sample(5, random_state=0)

In [ ]:
# Описание численных признаков
df[numerical_features].describe()

Можем видеть, что признак ``rain`` имеет малое стандартное отклонение и практически для всех объектов выборки имеет нулевое значение. 

In [ ]:
# Описание категориальных признаков
df[categorical_features].describe()

Категориальные признаки:
  - ``month`` - месяц, в формате строки.
  - ``day`` - день недели, в формате строки.

При препроцессинге данных необходимо перевести данные признаки в численный формат.

# Препроцесинг данных

In [ ]:
# Отделяем признаки и предсказываемое значение
X = df.drop("target", axis=1)
y = df["target"]
pprint(X)
pprint(y)

In [ ]:
X.sample(5, random_state=0)

In [ ]:
y.sample(5, random_state=0)

Преобразуем категориальные признаки в численный формат.

In [ ]:
# Кодирование признаков
encoder = OneHotEncoder(sparse_output=False)
X_encoded = encoder.fit_transform(X[categorical_features])

# Создание DataFrame c читаемыми именами кодированных признаков
encoded_features = encoder.get_feature_names_out(categorical_features)
X_encoded_df = pd.DataFrame(X_encoded, columns=encoded_features, index=X.index)
X_encoded_df.sample(5, random_state=0)

Делим выборку на обучающую и тестовую.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

print(f"Размер обучающей выборки составляет {len(X_train.index)} объектов.")
print(f"Размер тестовой выборки составляет {len(X_test.index)} объектов.")

In [ ]:
X_train.sample(5, random_state=0)

In [ ]:
y_train.sample(5, random_state=0)

Произведем нормировку численных признаков. 

'Обучание' ``scaler``'а производим на обучающей выборке, чтобы не вносить в модель информацию о предсказываемом значении.

Для координат нормировка представляет растяжение и перенос координат.

In [ ]:
# Обучаем 'scaler' на обучающей выборке
x_scaler = StandardScaler()
x_scaler.fit(X[numerical_features])

# Нормируем численные признаки во всей выборке
X_scaled = x_scaler.transform(X[numerical_features])
X_scaled_df = pd.DataFrame(X_scaled, columns=numerical_features, index=X.index)
X_scaled_df.sample(5, random_state=0)

Объединяем скалированные численные признаки и кодированные категориальные

In [ ]:
X_proc = np.hstack((X_scaled, X_encoded))
X_proc_df = pd.concat([X_scaled_df, X_encoded_df], axis=1)
X_proc_df.describe()

Далее скалируем значение ``target``

In [ ]:
# y = np.log(np.ones(len(y)) + y)
# y_scaled = (y - y.mean()) / y.std()
# y_proc_df = pd.DataFrame(y_scaled, columns=["target"], index=y.index)
y_proc_df = pd.DataFrame(np.log(np.ones(len(y)) + y), columns=["target"], index=y.index)
y_proc_df.describe()

Повторяем деление выборки на обучающую и тестовую. Множества объектов обучающей и тестовой выборок совпадают с полученными раннее, только теперь признаки в них отнормированны и закодированы.

In [ ]:
X_proc_train, X_proc_test, y_proc_train, y_proc_test = train_test_split(X_proc_df, y_proc_df, test_size=0.3, random_state=0)

In [ ]:
X_proc_train.sample(5, random_state=0)

In [ ]:
y_proc_train.sample(5, random_state=0)

In [ ]:
X_proc_train.describe()

In [ ]:
X_proc_test.describe()

In [ ]:
y_proc_train.describe()

In [ ]:
y_proc_test.describe()

Произведем анализ взаимной зависимости признаков.

In [ ]:
# sns.pairplot(X_scaled_df)

In [ ]:
y_proc_df_min = y_proc_df.min()[0]
y_proc_df_max = y_proc_df.max()[0]

y_step = round((y_proc_df_max - y_proc_df_min) / 10, 2)

for feature in X_proc_df[numerical_features].columns:
    x_min = min(X_proc_df[feature])
    x_max = max(X_proc_df[feature])
    x_step = (x_max - x_min) / 8
    
    plt.figure(figsize=[4,3])
    plt.xlim(x_min, x_max)

    plt.xticks(np.arange(x_min - x_step, x_max + 2 * x_step, x_step))
    plt.yticks(np.arange(y_proc_df_min - y_step, y_proc_df_max + 2 * y_step, y_step))
    
    plt.grid()
    plt.title(feature)
    plt.xlabel("feature value")
    plt.ylabel("target value")

    plt.scatter(X_proc_df[feature], y_proc_df)
    plt.show()

Из графиков зависимости предсказываемого значения от признаков, можно сделать следующие выводы:
  - Предсказывамое значение принимает большие значения лишь при нулевом значении ``rain``, вероятно существует зависимость предсказываемого значения от признака ``rain``.

# Эксперименты для предложенных методов

## Перцептрон

In [ ]:
# Параметры конструктора
MLPRegressor().get_params()

Список гиперпараметров, которые будут подбираться:
  - ``hidden_layer_sizes`` - количество и размер скрытых слоев
  - ``activation`` - функция активации
  - ``alpha`` - 'сила' L2-регуляризации
  - ``solver`` - использование классического SGD или оптимизации Adam

In [ ]:
# Размер скрытого слоя
hidden_layer_dims = [10, 50, 100, 150]
# Количество скрытых слоев
hidden_layer_nums = [1, 2, 3]

# Всевозможные комбинации вида 'num слоев размера dim'. 
hidden_layer_sizes = []
for num in hidden_layer_nums:
    for dim in hidden_layer_dims:
        hidden_layer_sizes.append([dim] * num)

# Сетка параметров
perceptron_param_grid = {
  'hidden_layer_sizes': hidden_layer_sizes, 
  'activation': ["logistic", "tanh", "relu"],
  'alpha': [0.0001, 0.001, 0.01, 0.1, 1.0, 5.0, 10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0],
  'solver': ["sgd", "adam"]
}
pprint(perceptron_param_grid)

Используем следующие метрики:
  - ``neg_mean_absolute_error`` - Mean absolute error regression loss
  - ``neg_root_mean_squared_error`` - Root mean squared error regression loss

In [ ]:
perceptron_metrics = {
    'neg_mean_absolute_error': mean_absolute_error,
    'neg_root_mean_squared_error': root_mean_squared_error
}

Модель - персептрон.

In [ ]:
perceptron = MLPRegressor(
  max_iter=20000, # Установим максимальное количество итераций больше дефолтного значения (1000)
  random_state=0 # Фиксируем seed
)

In [ ]:
perceptron_best = {}
perceptron_best_params = {}
perceptron_best_score = {}

# Отдельно ищем лучшие параметры для разных метрик
for scoring in perceptron_metrics.keys():
    grid_search = GridSearchCV(
      estimator=perceptron, 
      param_grid=perceptron_param_grid,
      scoring=scoring,
      n_jobs=-1
    )

    # Производим поиск лучших параметров
    grid_search.fit(X_proc_train, y_proc_train.values.ravel())

    # Запоминаем результаты: лучшая модель, её параметры и score
    perceptron_best[scoring] = grid_search.best_estimator_
    perceptron_best_params[scoring] = grid_search.best_params_
    perceptron_best_score[scoring] = grid_search.best_score_

    # Выводим результаты
    print(f"Scoring: {scoring}")
    print(f"Best score: {round(perceptron_best_score[scoring], 6)}")
    print("Best parameters:")
    pprint(perceptron_best_params[scoring])
    print("\n")

    # Результаты на тестовой выборке
    y_pred = perceptron_best[scoring].predict(X_proc_test)
    perceptron_test_score = - perceptron_metrics[scoring](y_proc_test, y_pred)
    print(f"Test score: {round(perceptron_test_score, 6)}")
    

Для сравнения полученных результатов с приведёнными в описании датасета и уже проведенных на нем экспериментов, произведем обратное преобразование ``y_pred`` и ``y_proc_test`` и посчитаем метрику на полученных данных.

In [ ]:
for scoring in perceptron_metrics.keys():
    print(f"Scoring: {scoring}")
    y_pred_trans = np.expm1(perceptron_best[scoring].predict(X_proc_test))
    svr_test_score = perceptron_metrics[scoring](np.expm1(y_proc_test), y_pred_trans)
    print(f"Test score for back-transformed: {round(svr_test_score, 6)}")

## Надарая-Ватсона

## SVR

Список гиперпараметров, которые будут подбираться:
  - ``kernel`` - функция ядра
  - ``gamma`` - коэффициент для ядерных функций ``rbf``, ``poly`` и ``sigmoid``
  - ``C`` - параметр регуляризации. Сила регуляризации обратно пропорциональна C. Должнен быть строго положительным. Penalty - l2 в квадрате

In [ ]:
# Сетка параметров
svr_param_grid = {
  'kernel': ["linear", "poly", "rbf", "sigmoid"],
  'gamma': ["scale", "auto"],
  'C': [0.001, 0.005, 0.1, 0.5, 1.0, 5.0, 10.0, 25.0, 50.0]
}
pprint(svr_param_grid)

Используем следующие метрики:
  - ``neg_mean_absolute_error`` - Mean absolute error regression loss
  - ``neg_root_mean_squared_error`` - Root mean squared error regression loss

In [ ]:
svr_metrics = {
    'neg_mean_absolute_error': mean_absolute_error,
    'neg_root_mean_squared_error': root_mean_squared_error
}

In [ ]:
svr_best = {}
svr_best_params = {}
svr_best_score = {}

# Отдельно ищем лучшие параметры для разных метрик
for scoring in svr_metrics.keys():
    grid_search = GridSearchCV(
      estimator=SVR(), 
      param_grid=svr_param_grid,
      scoring=scoring,
      n_jobs=-1
    )

    # Производим поиск лучших параметров
    grid_search.fit(X_proc_train, y_proc_train.values.ravel())
    
    svr_best[scoring] = grid_search.best_estimator_
    svr_best_params[scoring] = grid_search.best_params_
    svr_best_score[scoring] = grid_search.best_score_

    # Выводим результаты
    print(f"Scoring: {scoring}")
    print(f"Best score: {round(svr_best_score[scoring], 6)}")
    print("Best parameters:")
    pprint(svr_best_params[scoring])

    # Результаты на тестовой выборке
    y_test_pred = svr_best[scoring].predict(X_proc_test)
    svr_test_score = svr_metrics[scoring](y_proc_test, y_test_pred)
    print(f"Test score: {round(svr_test_score, 6)}")
    

Аналогично, произведем обратное преобразование y_pred и y_proc_test и посчитаем метрику на полученных данных.

In [ ]:
for scoring in svr_metrics.keys():
    print(f"Scoring: {scoring}")
    y_pred_trans = np.expm1(svr_best[scoring].predict(X_proc_test))
    svr_test_score = svr_metrics[scoring](np.expm1(y_proc_test), y_pred_trans)
    print(f"Test score for back-transformed: {round(svr_test_score, 6)}")

# Описание полученных результатов